In [38]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras import applications
from keras.models import Model

# 1 - Loading and preprocessing

In [39]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = '/home/thibault/Documents/TensorFlow/workspace/bjj_photos/train/'
validation_data_dir = '/home/thibault/Documents/TensorFlow/workspace/bjj_photos/test/'

In [40]:
##preprocessing
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

Found 247 images belonging to 2 classes.
Found 410 images belonging to 2 classes.


# 2 - Layers construction

In [41]:
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [42]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# 3 - Training

In [47]:
epochs = 5
train_samples = 2048
validation_samples = 832

In [48]:
model.fit_generator(
        train_generator,
        steps_per_epoch=train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_samples// batch_size,)
#About 60 seconds an epoch when using CPU

Epoch 1/5
64/64 [==============================] - 99s - loss: 0.2118 - acc: 0.9219 - val_loss: 1.0267 - val_acc: 0.6951
Epoch 2/5
64/64 [==============================] - 110s - loss: 0.0635 - acc: 0.9801 - val_loss: 1.8387 - val_acc: 0.6841
Epoch 3/5
64/64 [==============================] - 91s - loss: 0.0239 - acc: 0.9930 - val_loss: 2.5506 - val_acc: 0.6829
Epoch 4/5
64/64 [==============================] - 76s - loss: 0.0081 - acc: 0.9969 - val_loss: 4.0207 - val_acc: 0.6341
Epoch 5/5
64/64 [==============================] - 69s - loss: 0.0492 - acc: 0.9878 - val_loss: 2.5049 - val_acc: 0.6780


In [49]:
model.save_weights('/home/thibault/Documents/TensorFlow/workspace/weight_bjj/basic_cnn_30_epochs.h5')

# 4 - Evaluation

In [50]:
model.evaluate_generator(validation_generator, validation_samples)

[2.4452584802173076, 0.68052591491127157]

In [51]:
model.metrics_names

['loss', 'acc']